Transpose / Pivot table Pandas

In [ ]:
import pandas as pd

df.set_index('Material Name', inplace=True)

trans = df.transpose()

trans.reset_index(inplace=True)

trans = trans[cols]

# One way to do it, for now it can not remain same order easily.
trans.melt(id_vars=["index"], ignore_index = False).to_excel('../output/hope.xlsx')

# Another way but preverse order.
result= trans.set_index(["index"]).stack()
# Or
result = df.pivot(index=["some cols"], columns="cols contain values to set name of columns", values="cols that want to get values from")

Convert to datetime

In [ ]:
df['columns name'] = pd.to_datetime(df['columns name']], format="%d/%m/%Y", yearfirst=True)

Dataframe to SQL

In [ ]:
from sqlalchemy import create_engine

df = pd.read_excel('Byun/take_in.xlsx', header=10)

sql_engine = create_engine(
        'mysql+pymysql://admin:password@ip/scheme', pool_recycle=3600
    )
take_in.to_sql('table', sql_engine,
              if_exists='replace')

Walk through files in folder

In [ ]:
import pandas as pd
import numpy as np
from os import walk

pd.set_option('display.max_columns', 30)
pd.set_option('display.max_rows', 40)
path = '../data/bom'
file_names = next(walk(path), (None, None, []))[2]
file_names = (f'{path}/{x}' for x in file_names)
dfs = [pd.read_excel(x, header=0, usecols=['Unnamed: 0', 'Plant', 'Entry Date', 'Material', 'Location',
       'Stock Type', 'Movement Type', 'Posting Date', 'Unit', 'Quantity',
       'Material Document No', 'Provide', 'Reference', 'Unnamed: 13',
       'Unnamed: 14', 'Unnamed: 15', 'Unnamed: 16', 'Previous Stock',
       'Entry User']) for x in file_names]
def loc_df(df):
    lst = [x[0] for x in df['Movement Type'].str.split()]
    df['movement type check'] = lst
    df = df.loc[df['movement type check'].isin(['101','102','103'])]
    return df

dfs = [loc_df(df) for df in dfs]

result = pd.concat(dfs)
result.columns
result = result[['Material', 'Unit', 'Movement Type', 'Quantity', 'Reference']]
end = result.pivot_table(index=['Material', 'Unit'], columns=['Reference'], aggfunc=np.sum)
end.to_excel('../output/GIANGALO.xlsx')



Concat all sheets in file

In [ ]:
import pandas as pd
import numpy as np
dfs = pd.read_excel('../data/huy_data/TK 1571-2021 NGUYET GOI CHI THUY (1).xlsx', sheet_name=None, header=5,
    #    usecols=['TT', 'Unnamed: 1', 'Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4', 'USD',
    #    'Số lượng', 'Số lượng.1', 'Số lượng.2', 'Số lượng.3']
)
# for name, df in dfs.items():
#     print(df.columns)

# def loc_df(df):
#     df = df.loc[~(pd.isna(df['Unnamed: 3']))]
#     return df
dfs = list(dfs.values())
# dfs = [loc_df(df) for df in dfs]
result = pd.concat(dfs)
result.to_excel('../output/TK 1571-2021 NGUYET GOI CHI THUY.xlsx')


Concat BOM in folder

In [ ]:
import pandas as pd
import numpy as np
from os import walk

pd.set_option('display.max_columns', 30)
pd.set_option('display.max_rows', 40)
path = '../data/huy_data/HĐ 2021'
file_names = next(walk(path), (None, None, []))[2]
file_names = (f'{path}/{x}' for x in file_names)

dfs = [pd.read_excel(x, header=10
       ) for x in file_names]
# for x in dfs:
#     print(x.columns)

def loc_df(df):
    df = df.loc[~pd.isna(df['(5)'])]
    return df

dfs = [loc_df(df) for df in dfs]
result = pd.concat(dfs)
result.ffill(inplace=True)
# result.to_excel('../output/HĐ 2021.xlsx')


In [ ]:
result.set_index(['(1)','(2)', '(3)','(4)']).stack().to_excel('../output/HĐ 2021.xlsx')

In [ ]:
import pandas as pd
import numpy as np
from os import walk

pd.set_option('display.max_columns', 30)
pd.set_option('display.max_rows', 40)
path = '../data/GIANG'
file_names = next(walk(path), (None, None, []))[2]
file_names = (f'{path}/{x}' for x in file_names)
dfs = [pd.read_excel(x, header=1, 
            # usecols=['Unnamed: 0', 'Plant', 'Entry Date', 'Material', 'Location',
            # 'Stock Type', 'Movement Type', 'Posting Date', 'Unit', 'Quantity',
            # 'Material Document No', 'Provide', 'Reference', 'Unnamed: 13',
            # 'Unnamed: 14', 'Unnamed: 15', 'Unnamed: 16', 'Previous Stock',
            # 'Entry User']
            ) 
            for x in file_names
       ]
# def loc_df(df):
#     lst = [x[0] for x in df['Movement Type'].str.split()]
#     df['movement type check'] = lst
#     df = df.loc[df['movement type check'].isin(['101','102','103'])]
#     return df

# dfs = [loc_df(df) for df in dfs]

result = pd.concat(dfs)
result.to_excel('../output/GIANGALO.xlsx')



In [ ]:
import pandas as pd
from pandas import ExcelWriter


# df = pd.read_excel('../data/COSTING BOM T1-12.xlsx', header=5)

dfs = pd.read_excel('../data/COSTING BOM T1-12.xlsx', sheet_name=None, header=5,)

def loc_df(df):
    columns = df.columns
    columns = [column for column in columns if not str(column).startswith('Unnamed')]
    df = df[columns]
    df = df.loc[~pd.isna(df['Mã']) & ~(df['Mã'] == 'Đơn giá NVL') & ~(df['Mã'] == 0)]
    result= df.set_index(["Mã"]).stack()

    result = pd.DataFrame(result)
    result.reset_index(inplace=True)
    result = result.drop(result[result[0] == 0].index)
    return result

dfs = [loc_df(df) for df in dfs.values()]

result = pd.concat(dfs)

with ExcelWriter('../output/end.xlsx') as writer:
    for n, df in enumerate(dfs, start=1):
        df.rename(columns={
            'Mã': 'TP',
            'level_1': 'NVL',
            0: 'VALUE'
        }, inplace=True)
        df.to_excel(writer,'sheet %s' % n, index=False)


GIANG

In [ ]:
import pandas as pd

df = pd.read_excel('../data/BOM INZI.xlsb')

df = df.loc[~pd.isna(df['Semi']) & ~pd.isna(df['Technical BOM'])]
df = df[['Level.', 'Product', 'Semi', 'RM', 'Name', 'Unit','Technical BOM']]
df['Level.'] = [x.split('.')[-1] for x in df['Level.']]
df['Level.'] = df['Level.'].astype(int)

# rats = []
# previous_level = 1
# ratio = [1] * (len(df['Level.'].unique()) + 1)

# for row in zip(df['Level.'], df['Technical BOM']):
#     if row[0] > 1:
#         if row[0] > previous_level:
#             ratio[row[0]] = ratio[row[0] - 1] * previous_ratio
    
#     rats.append(ratio[row[0]])

#     previous_ratio = row[1]
#     previous_level = row[0]

# df['ratio'] = rats
# df['BOM'] = df['Technical BOM'] * df['ratio']



indexs = []
current_level = 1
for count, x in enumerate(df['Level.'], start=-1):
    if x > current_level:
        indexs.append(count)
    current_level = x

df = df.drop(df.index[indexs])

df.to_excel('../output/ENDING.xlsx')



In [ ]:
import pandas as pd

df = pd.read_excel('../data/BCQT2021.BOM.xlsx', header=9)
df = df.groupby(['Unnamed: 1', 'Unnamed: 2', 'Unnamed: 3', 'Mã', 'Tên',
       'Đơn vị tính']).sum()
df.to_excel('../output/bcqt.xlsx')


In [ ]:
import pandas as pd

df = pd.read_excel('../data/(03.04.2022)_MEGAELEC_16 FORM_PROCESSING.xlsb', header=10)
df = df.groupby(['(1)', '(2)', '(3)', '(4)', '(5)', '(6)', '(7)']).sum()
df.reset_index().to_excel('../output/GIANGx.xlsx')
print(df.shape[0])

In [ ]:
import pandas as pd

def Insert_row(row_number, df, row_value):
    # Starting value of upper half
    start_upper = 0
  
    # End value of upper half
    end_upper = row_number
  
    # Start value of lower half
    start_lower = row_number
  
    # End value of lower half
    end_lower = df.shape[0]
  
    # Create a list of upper_half index
    upper_half = [*range(start_upper, end_upper, 1)]
  
    # Create a list of lower_half index
    lower_half = [*range(start_lower, end_lower, 1)]
  
    # Increment the value of lower half by 1
    lower_half = [x.__add__(1) for x in lower_half]
  
    # Combine the two lists
    index_ = upper_half + lower_half
  
    # Update the index of the dataframe
    df.index = index_
  
    # Insert a row at the end
    df.loc[row_number] = row_value
   
    # Sort the index labels
    df = df.sort_index()
  
    # return the dataframe
    return df
  


df = pd.read_excel('../data/(01.04.2022)_MEGAELEC_16 FORM_PROCESSING copy (1).xlsb', header=11)
k = 0

df = df.loc[df['(5)'] != 'EX-04-00017']
current = '00-000-2276-REV-E-MG'
for count, x in enumerate(df['(2)']):
    if x != current:
        for _ in range(11):
            df = Insert_row(count + k, df, pd.Series())
        k += 11
    current = x
    

df.to_excel('../output/aaaaa.xlsx')


In [ ]:
import pandas as pd

df = pd.read_excel('../data/(04.04.2022)_MEGAELEC_SUMMARIZE REPORT.xlsx')

In [ ]:
df = df[['ECUS CODE', 'ACC CODE', 'DECRIPTION']]

In [ ]:
def get_code_step_1(row):
    string = row['DECRIPTION']
    string = string.split()
    for x in string:
        if len(x.split('-')) > 1:
            return x.split(',')[0]
    return row['DECRIPTION']

def get_code_step_2(row):
    string = row['CODE']
    if len(string.split()) > 1:
        return string.split(',')[0].split()[-1]
    return row['CODE']

df['CODE'] = df.apply(lambda row: get_code_step_1(row), axis=1)
df['CODE'] = df.apply(lambda row: get_code_step_2(row), axis=1)

df_obj = df.select_dtypes(['object'])

df[df_obj.columns] = df_obj.apply(lambda x: x.str.strip())
df.to_excel('../output/result.xlsx')



In [ ]:
import pandas as pd
import numpy as np

bom = pd.read_excel('../data/bom.xlsx')
iob = pd.read_excel('../data/iob.xlsx')

In [ ]:
print(bom.columns)
print(iob.columns)

In [ ]:
result = pd.merge(iob, bom, how='inner', left_on=['balance__code', 'balance__unit'], right_on=['bom_belong__code', 'bom_belong__unit'])

In [ ]:
import pandas as pd
import numpy as np
from os import walk

'''  MATL '''
pd.set_option('display.max_columns', 30)
pd.set_option('display.max_rows', 40)
path = '../data/2021'
tp_url = '../data/DAILY PRODUCTION 2021.xlsx'
sheet_name = '2021'
output_url = '../output/2021_full.xlsx'

file_names = next(walk(path), (None, None, []))[2]
file_names = (f'{path}/{x}' for x in file_names)
dfs = [pd.read_excel(x, header=0, converters={'Article No': str, 'Material Code': str, 'File No': str}, usecols=['Article No', 'Material Code', 'Unit', 'D/O Unit Stock Qty']) for x in file_names]

result = pd.concat(dfs)
df = result
nvl = df.groupby(['Article No', 'Material Code', 'Unit'], sort=False).agg(
            {'D/O Unit Stock Qty': 'sum'})
nvl.reset_index(inplace=True)
tp = pd.read_excel(tp_url, sheet_name=sheet_name, converters={'Article Code': str}, usecols=['Article Code', 'TOTAL OUTPUT Qty.'])
sx = tp.groupby(['Article Code'], sort=False).agg(
            {'TOTAL OUTPUT Qty.': 'sum'})
sx.reset_index(inplace=True)
nvl.rename(columns={'Article No': 'Article Code'}, inplace=True)


bom = pd.merge(nvl, sx, how="left", on='Article Code', sort=False,
                          indicator=False, validate=None)
bom['BOM'] = bom['D/O Unit Stock Qty'] / bom['TOTAL OUTPUT Qty.']
# bom.loc[~pd.isna(bom['TOTAL OUTPUT Qty.'])].to_excel('../output/2019.xlsx')
bom.to_excel(output_url)




In [ ]:
import pandas as pd
import numpy as np
from os import walk

'''  MATL '''
pd.set_option('display.max_columns', 30)
pd.set_option('display.max_rows', 40)
path = '../data/bom btp'

file_names = next(walk(path), (None, None, []))[2]
file_names = (f'{path}/{x}' for x in file_names)
dfs = [pd.read_excel(x, header=0, converters={'Material': str, 'Component': str}) for x in file_names]

result = pd.concat(dfs)

result.to_excel('../output/semi.xlsx')


In [ ]:
import pandas as pd
import numpy as np

In [ ]:
name_df = pd.read_excel('../data/2022-05-05_BOM_2.xlsx', converters={'Code': str, 'Material Code': str})
name_df = name_df[['Code', 'ECUS']].drop_duplicates()
bom = pd.read_excel('../data/Bom.xlsx', converters={'Code': str, 'Material Code': str})
result = pd.merge(bom, name_df, on='Code', how='left')
result.sort_values(['Code', 'ECUS'], ascending=[True, True], inplace=True)
result.to_excel('../output/sort.xlsx')

In [ ]:
sx = pd.read_excel('../data/Truy van ket qua san xuat 01012022 31032022.xlsx', header=2)
nvl = pd.read_excel('../data/Tinh hinh nhap NVL vao sx 01012022 31032022 - Copy.xlsx')

sx.rename(columns={'Đơn vị': 'Product Unit', 'Mã hàng': 'Product'}, inplace=True)
nvl.rename(columns={'Đơn vị': 'Material Unit', 'Mã số NPL': 'Material'}, inplace=True)

sx = sx[['Product', 'LotNo', 'Số lượng hàng đạt', 'Product Unit']]
nvl = nvl[['Material', 'LotNo', 'Số lượng nhập', 'Material Unit']]

sx.drop_duplicates(inplace=True)
result = pd.merge(sx, nvl, how="inner", on='LotNo', sort=False,
                  indicator=False, validate=None)

pro_qty = result[['Product', 'LotNo', 'Số lượng hàng đạt']]
pro_qty.drop_duplicates(inplace=True)
pro_qty = pro_qty.groupby('Product').agg(
    {'Số lượng hàng đạt': 'sum'})
pro_qty.reset_index(inplace=True)
result = result.groupby(['Product', 'Product Unit', 'Material', 'Material Unit']).agg(
    {'Số lượng nhập': 'sum'})
result.reset_index(inplace=True)
result = pd.merge(result, pro_qty, how="left", on=['Product'], sort=False,
                  indicator=False, validate=None)
result['BOM'] = result['Số lượng nhập'] / result['Số lượng hàng đạt']
result.to_excel('../output/bom.xlsx')

In [ ]:

result = df.groupby(['(2)', '(4)', '(5)', '(7)', '(9)']).agg(
    {'(8)': 'sum'})
result.reset_index(inplace=True)
result['(9)'] = result['(9)'].str.replace('a', '')
result.to_excel('../output/bom.xlsx')

In [ ]:
df = pd.read_excel('../output/bom.xlsx')

df[' Mã NVL'] = df[' Mã NVL'].str.upper()
df['Đơn vị tính.1'] = df['Đơn vị tính.1'].str.strip()

df[' Mã NVL'] = df[' Mã NVL'].str.upper()
df['Đơn vị tính.1'] = df['Đơn vị tính.1'].str.strip()

df = df[[' Mã NVL', 'Đơn vị tính.1']]
df = df.drop_duplicates(keep='first')
df = df.loc[df.duplicated(subset=' Mã NVL', keep=False)]


In [ ]:
df.to_excel('../output/dup.xlsx')

In [ ]:
import pandas as pd
bare = pd.read_excel('../data/merge_pros.xlsx', sheet_name='bare')
df1 = pd.read_excel('../data/merge_pros.xlsx', sheet_name='first', converters={'cd_import': str})
df2 = pd.read_excel('../data/merge_pros.xlsx', sheet_name='second', converters={'cd_export': str})

result = pd.merge(df1, df2, how="outer", on=['code', 'registered_date'], sort=False, indicator=False, validate=None).sort_values(['code', 'registered_date'])

count_1 = df1.groupby(['code', 'registered_date']).aggregate({'cd_import': 'count'}).reset_index()
count_2 = df2.groupby(['code', 'registered_date']).aggregate({'cd_export': 'count'}).reset_index()
count = pd.merge(count_1, count_2, how="outer", on=['code', 'registered_date'], sort=False, indicator=False, validate=None).sort_values(['code', 'registered_date'])

count = count.fillna(1)
count['total'] = count['cd_import'] * count['cd_export']

count['cd_import'] = count['cd_import'].astype(int)
count['cd_export'] = count['cd_export'].astype(int)
count['total'] = count['total'].astype(int)


In [ ]:
indexes_to_delete = list()

i = 0
for row in zip(count['cd_import'], count['cd_export'], count['total']):
    if row[0] > 1 and row[1] > 1:
        if row[0] >= row[1]:
            k = row[1] - 1
        else:
            k = row[0] - 1
        l = i + 1
        for _ in range(k):
            indexes_to_delete += range(l, l + row[1])
            l += row[1]
    i += row[2]

    

In [ ]:
import pandas as pd

df = pd.read_excel('../data/BOM.CT.xlsx', sheet_name='Sheet1', header=2)

df['(2)'] = df['(2)'].str.upper()
df['(4)'] = df['(4)'].str.upper()
df['(5)'] = df['(5)'].str.upper()
df['(7)'] = df['(7)'].str.upper()

df['(2)'] = df['(2)'].str.strip()
df['(4)'] = df['(4)'].str.strip()
df['(5)'] = df['(5)'].str.strip()
df['(7)'] = df['(7)'].str.strip()

result = df.groupby(['(2)', '(4)', '(5)', '(7)']).agg(
    {'(8)': 'sum'})
result.reset_index(inplace=True)
name_tp = df[['(2)', '(3)']]
name_tp.drop_duplicates(inplace=True, keep='first', subset='(2)')
name_nvl = df[['(5)', '(6)']]
name_nvl.drop_duplicates(inplace=True, keep='first', subset='(5)')
result = pd.merge(result, name_tp, on='(2)', how='left')
result = pd.merge(result, name_nvl, on='(5)', how='left')
result.to_excel('../output/bom.xlsx')


In [ ]:
import pandas as pd

In [ ]:
bom_df = pd.read_excel('../data/BOM_TC.xlsx', sheet_name='Sheet1', header=2)
semi_1 = pd.read_excel('../data/2000185.XLSX', sheet_name='ST-0,075x7,212')
semi_2 = pd.read_excel('../data/2000185.XLSX', sheet_name='ST-0,075x5,744')

bom_df['(5)'] = bom_df['(5)'].str.upper()
semi_1['Products code'] = semi_1['Products code'].str.upper()
semi_2['Products code'] = semi_2['Products code'].str.upper()
semi_1 = semi_1[['Products code', 'Product unit', 'Ecus (Vlookup)','Component Description', 'Qty', 'Unit']]
semi_2 = semi_2[['Products code', 'Product unit', 'Ecus (Vlookup)','Component Description', 'Qty', 'Unit']]

result = pd.merge(bom_df, semi_1, left_on=['(5)', '(7)'], right_on=['Products code', 'Product unit'], how='left' ,sort=False, indicator=False, validate=None)
result = pd.merge(result, semi_2, left_on=['(5)', '(7)'], right_on=['Products code', 'Product unit'], how='left' ,sort=False, indicator=False, validate=None)

def replace_material(row):
    if not pd.isna(row['Ecus (Vlookup)_x']):
        return row['Ecus (Vlookup)_x']
    if not pd.isna(row['Ecus (Vlookup)_y']):
        return row['Ecus (Vlookup)_y']
    return row['(5)']

def replace_name(row):
    if not pd.isna(row['Component Description_x']):
        return row['Component Description_x']
    if not pd.isna(row['Component Description_y']):
        return row['Component Description_y']
    return row['(6)']

def replace_unit(row):
    if not pd.isna(row['Unit_x']):
        return row['Unit_x']
    if not pd.isna(row['Unit_y']):
        return row['Unit_y']
    return row['(7)']

def replace_bom(row):
    if not pd.isna(row['Qty_x']):
        return row['Qty_x'] * row['(8)']
    if not pd.isna(row['Qty_y']):
        return row['Qty_y'] * row['(8)']
    return row['(8)']


result['(5)'] = result.apply(lambda row: replace_material(row), axis=1)
result['(6)'] = result.apply(lambda row: replace_name(row), axis=1)
result['(7)'] = result.apply(lambda row: replace_unit(row), axis=1)
result['(8)'] = result.apply(lambda row: replace_bom(row), axis=1)


In [ ]:
df = result

df['(2)'] = df['(2)'].str.upper()
df['(4)'] = df['(4)'].str.upper()
df['(5)'] = df['(5)'].str.upper()
df['(7)'] = df['(7)'].str.upper()

df['(2)'] = df['(2)'].str.strip()
df['(4)'] = df['(4)'].str.strip()
df['(5)'] = df['(5)'].str.strip()
df['(7)'] = df['(7)'].str.strip()

result = df.groupby(['(2)', '(4)', '(5)', '(7)']).agg(
    {'(8)': 'sum'})
result.reset_index(inplace=True)
name_tp = df[['(2)', '(3)']]
name_tp.drop_duplicates(inplace=True, keep='first', subset='(2)')
name_nvl = df[['(5)', '(6)']]
name_nvl.drop_duplicates(inplace=True, keep='first', subset='(5)')
result = pd.merge(result, name_tp, on='(2)', how='left')
result = pd.merge(result, name_nvl, on='(5)', how='left')
result.to_excel('../output/boms.xlsx')


In [ ]:
import pandas as pd

df = pd.read_excel('../data/abc.xlsx', converters={'cd_import': str, 'cd_export': str})

df["import_quantity"] = pd.to_numeric(df["import_quantity"])
df["export_quantity"] = pd.to_numeric(df["export_quantity"])
df["begin"] = 0
df['temp_import_quantity'] = df.groupby('code')['import_quantity'].cumsum()
df['temp_export_quantity'] = df.groupby('code')['export_quantity'].cumsum()
df['end'] = df['begin'] + df['temp_import_quantity'] - df['temp_export_quantity']
df['begin'] = df['end'] + df['export_quantity'] - df['import_quantity']

In [ ]:
def datetime_to_str(row):
    try:
        return row['Ngày ĐK'].strftime('%d/%m/%Y')
    except:
        return row['Ngày ĐK']

In [ ]:
import pandas as pd

bom = pd.read_excel('../data/bom.xlsx')
ecus = pd.read_excel('../data/ecus.xlsx', converters={'Số TK': str})

ecus['Ngày ĐK'] = ecus.apply(lambda row: datetime_to_str(row), axis=1)
ecus['Ngày ĐK'] = pd.to_datetime(ecus['Ngày ĐK'], dayfirst=True)

name_df = ecus[['Mã NPL/SP', 'Tên hàng', 'Đơn vị tính']]
name_df.drop_duplicates(inplace=True, keep='first', subset=['Mã NPL/SP'])

import_df = ecus.loc[(ecus['Mã loại hình'] == 'A12') & (~pd.isna(ecus['Mã NPL/SP']))]
export_df = ecus.loc[(ecus['Mã loại hình'] == 'E62') & (~pd.isna(ecus['Mã NPL/SP']))]


import_df = import_df[['Số TK', 'Ngày ĐK', 'Tổng số lượng', 'Tên hàng', 'Mã NPL/SP', 'Đơn vị tính', 'Tổng trị giá', 'Thuế suất XNK']]
export_df = export_df[['Số TK', 'Ngày ĐK', 'Tổng số lượng', 'Tên hàng', 'Mã NPL/SP', 'Đơn vị tính']]
export_bom_df = pd.merge(export_df, bom, how='left', left_on='Mã NPL/SP', right_on='product_code')

import_df.rename(columns={
    'Số TK': 'CD IMPORT',
    'Mã NPL/SP': 'NVL',
    'Tên hàng': 'NVL NAME',
    'Đơn vị tính': 'NVL UNIT',
    'Tổng số lượng': 'NVL QTY'
}, inplace=True)
export_bom_df.rename(columns={
    'Số TK': 'CD EXPORT',
    'Mã NPL/SP': 'TP',
    'Tên hàng': 'TP NAME',
    'Đơn vị tính': 'TP UNIT',
    'Tổng số lượng': 'TP QTY',
    'code': 'NVL'
}, inplace=True)

export_bom_df['TP QTY'] = export_bom_df['TP QTY'] * export_bom_df['bom_value']

df = pd.merge(import_df, export_bom_df, how="outer", on=['NVL', 'Ngày ĐK'], sort=False, indicator=False, validate=None)
df = df.loc[df['NVL'].isin(import_df['NVL'].unique())]
df.sort_values(['NVL', 'Ngày ĐK'], inplace=True)

count_1 = import_df.groupby(['NVL', 'Ngày ĐK']).aggregate({'CD IMPORT': 'count'}).reset_index()
count_2 = export_bom_df.groupby(['NVL', 'Ngày ĐK']).aggregate({'CD EXPORT': 'count'}).reset_index()
count_2 = count_2.loc[count_2['NVL'].isin(import_df['NVL'].unique())]
count = pd.merge(count_1, count_2, how="outer", on=['NVL', 'Ngày ĐK'], sort=False, indicator=False, validate=None).sort_values(['NVL', 'Ngày ĐK'])
count = count.fillna(1)
count['total'] = count['CD IMPORT'] * count['CD EXPORT']
count['CD IMPORT'] = count['CD IMPORT'].astype(int)
count['CD EXPORT'] = count['CD EXPORT'].astype(int)
count['total'] = count['total'].astype(int)
count = count.loc[count['NVL'].isin(import_df['NVL'].unique())]
indexes_to_delete = list()

i = 0
for row in zip(count['CD IMPORT'], count['CD EXPORT'], count['total']):
    if row[0] > 1 and row[1] > 1:
        k = row[0] - 1
        if row[0] >= row[1]:
            k = row[1] - 1
        l = i + 1
        for _ in range(k):
            indexes_to_delete += range(l, l + row[1])
            l += row[1]
    i += row[2]

df = df.drop(indexes_to_delete)

# 4 columns to insert
import_cd = list()
import_qt = list()
export_cd = list()
export_qty = list()
current_import_cd = ''
current_export_cd = ''
current_code = None
current_date = ''
current_tp = None
for _, row in df.iterrows():
    if current_code:
        if current_import_cd == row['CD IMPORT'] and current_import_cd:
            if current_date == row['Ngày ĐK'] and current_code == row['NVL']:
                import_cd.append('')
                import_qt.append(0)
            else:
                import_cd.append(row['CD IMPORT'])
                import_qt.append(row['NVL QTY'])
        else:
            import_cd.append(row['CD IMPORT'])
            import_qt.append(row['NVL QTY'])

        if current_export_cd == row['CD EXPORT'] and current_export_cd:
            if current_date == row['Ngày ĐK'] and current_code == row['NVL'] and current_tp==row['TP']:
                export_cd.append('')
                export_qty.append(0)
            else:
                export_cd.append(row['CD EXPORT'])
                export_qty.append(row['TP QTY'])
        else:
            export_cd.append(row['CD EXPORT'])
            export_qty.append(row['TP QTY'])
    else:
        import_cd.append(row['CD IMPORT'])
        import_qt.append(row['NVL QTY'])
        export_cd.append(row['CD EXPORT'])
        export_qty.append(row['TP QTY'])

    current_code = row['NVL']
    current_date = row['Ngày ĐK']
    current_import_cd = row['CD IMPORT']
    current_export_cd = row['CD EXPORT']
    current_tp = row['TP']


df['CD IMPORT'] = import_cd
df['CD EXPORT'] = export_cd
df['NVL QTY'] = import_qt
df['TP QTY'] = export_qty

# df.loc[df['NVL'] == '3216-02540-00'].to_excel('../output/wtf.xlsx')


In [ ]:
out_first = df.drop_duplicates(subset=['NVL'], keep='first')
out_first = out_first.loc[pd.isna(out_first['NVL QTY'])]

In [ ]:
df.columns

In [3]:
import pandas as pd

df = pd.read_excel('../output/prepare_move.xlsx', converters={'import_cd': str, 'export_cd': str})
df['export_date'] = df['Ngày ĐK']
df = df.loc[~pd.isna(df['import_cd']) | ~pd.isna(df['export_cd'])]
df.rename(columns={'Ngày ĐK': 'import_date'}, inplace=True)
df.reset_index(inplace=True)
del df['index']

export_null = 0
import_null = 0

nvl_names = []
nvl_units = []

import_cds = []
import_dates = []
import_qty = []

tax_totals = []
tax_rates = []

export_cds = []
export_dates = []
export_qty = []
tps = []
tp_units = []
tp_names = []
bom_values = []

to_delete = []

current_code = None

for count, row in df.iterrows():
    if not current_code:
        if not pd.isna(row['import_cd']):
            import_cds.append(row['import_cd'])
            nvl_names.append(row['nvl_name_x'])
            nvl_units.append(row['nvl_unit_x'])
            import_dates.append(row['import_date'])
            import_qty.append(row['import_qty'])
            tax_totals.append(row['Tổng trị giá'])
            tax_rates.append(row['Thuế suất XNK'])
        else:
            import_null += 1

        if not pd.isna(row['export_cd']):
            export_cds.append(row['export_cd'])
            export_dates.append(row['export_date'])
            export_qty.append(row['export_qty'])
            tps.append(row['tp_code'])
            tp_units.append(row['tp_unit'])
            tp_names.append(row['tp_name'])
            bom_values.append(row['bom'])

        else:
            export_null += 1

    else:
        if current_code == row['nvl_code']:
            if not pd.isna(row['import_cd']):
                import_cds.append(row['import_cd'])
                nvl_names.append(row['nvl_name_x'])
                nvl_units.append(row['nvl_unit_x'])
                import_dates.append(row['import_date'])
                import_qty.append(row['import_qty'])
                tax_totals.append(row['Tổng trị giá'])
                tax_rates.append(row['Thuế suất XNK'])
            else:
                import_null += 1

            if not pd.isna(row['export_cd']):
                export_cds.append(row['export_cd'])
                export_dates.append(row['export_date'])
                export_qty.append(row['export_qty'])
                tps.append(row['tp_code'])
                tp_units.append(row['tp_unit'])
                tp_names.append(row['tp_name'])
                bom_values.append(row['bom'])
            else:
                export_null += 1
    
        else:
            j = min([import_null, export_null])
            to_delete += range(count-j, count)

            import_cds += [None] * import_null
            nvl_names += [None] * import_null
            nvl_units += [None] * import_null
            import_dates += [None] * import_null
            import_qty += [None] * import_null
            tax_totals += [None] * import_null
            tax_rates += [None] * import_null

            export_cds += [None] * export_null
            export_dates += [None] * export_null
            export_qty += [None] * export_null
            tps += [None] * export_null
            tp_units += [None] * export_null
            tp_names += [None] * export_null
            bom_values += [None] * export_null

            import_null = 0
            export_null = 0

            if pd.isna(row['import_cd']):
                import_null = 1
            else:
                import_cds.append(row['import_cd'])
                nvl_names.append(row['nvl_name_x'])
                nvl_units.append(row['nvl_unit_x'])
                import_dates.append(row['import_date'])
                import_qty.append(row['import_qty'])
                tax_totals.append(row['Tổng trị giá'])
                tax_rates.append(row['Thuế suất XNK'])

            if pd.isna(row['export_cd']):
                export_null = 1
            else:
                export_cds.append(row['export_cd'])
                export_dates.append(row['export_date'])
                export_qty.append(row['export_qty'])
                tps.append(row['tp_code'])
                tp_units.append(row['tp_unit'])
                tp_names.append(row['tp_name'])
                bom_values.append(row['bom'])

    current_code = row['nvl_code']

export_cds.append(None)
export_dates.append(None)
export_qty.append(None)
tps.append(None)
tp_units.append(None)
tp_names.append(None)
bom_values.append(None)

df['import_cd'] = import_cds
df['export_cd'] = export_cds
df['nvl_name_x'] = nvl_names
df['nvl_unit_x'] = nvl_units
df['import_date'] = import_dates
df['import_qty'] = import_qty
df['Tổng trị giá'] = tax_totals
df['Thuế suất XNK'] = tax_rates
df['export_date'] = export_dates
df['export_qty'] = export_qty
df['tp_code'] = tps
df['tp_unit'] = tp_units
df['tp_name'] = tp_names
df['bom'] = bom_values

df = df.drop(to_delete)



ValueError: Length of values (2381) does not match length of index (2390)

In [ ]:
current_code = None

current_import_cd = []
current_export_cd = []

current_import = []
current_export = []


import_cds = []
export_cds = []

codes = []
dfx = df
for count, row in dfx.iterrows():
    if not current_code:
        current_import_cd.append(row['CD IMPORT'])
        current_export_cd.append(row['CD EXPORT'])

        current_import.append(row['NVL QTY'])
        current_export.append(row['TP QTY'])

    else:
        if current_code == row['NVL']:
            current_import_cd.append(row['CD IMPORT'])
            current_export_cd.append(row['CD EXPORT'])
            
            current_import.append(row['NVL QTY'])
            current_export.append(row['TP QTY'])
        else:
            # GET DICE FOR DEVIDE LIST
            k = 0
            im = current_import[k]
            ex = 0
            dice = []
            for i in range(len(current_export)):
                ex += current_export[i]
                if ex >= im:
                    dice.append(i)
                    k += 1
                    if len(current_export) == 1:
                        pass
                    else:
                        im += current_import[k]


            temp_cd = []
            l = len(current_import_cd)
            if len(dice) > 0:
                for i in range(len(dice) - 1):
                    temp_cd.append(current_import_cd.pop(0))
                    if i > 0:
                        n = dice[i] - dice[i-1] - 1
                    else:
                        n = dice[i] - 1
                    temp_cd += [None] * n
            else:
                temp_cd += current_import_cd
            
            if not [x for x in current_export_cd if x]:
                codes.append(current_code)
            
            # if l - len(temp_cd) < 0:
            #     print(current_code, dice)
            # temp_cd += [None] * (l - len(temp_cd))
            import_cds += temp_cd

            current_import = []
            current_export = []
            current_import_cd = []
            current_export_cd = []

            current_import_cd.append(row['CD IMPORT'])
            current_export_cd.append(row['CD EXPORT'])
            current_import.append(row['NVL QTY'])
            current_export.append(row['TP QTY'])

    current_code = row['NVL']


In [ ]:
dfx = dfx.loc[~dfx['NVL'].isin(codes)]


In [ ]:
dfx.fillna(0, inplace=True)
dfx.reset_index(inplace=True)
del dfx['index']

current_code = None

current_import_cd = []
current_export_cd = []

current_import = []
current_export = []

nvl_names = []
nvl_units = []
import_cds = []
import_dates = []
import_qty = []
tax_totals = []
tax_rates = []


export_cds = []
export_dates = []
export_qty = []
tps = []
tp_units = []
tp_names = []
bom_values = []

to_delete = []


for count, row in dfx.iterrows():
    if not current_code:
        current_import_cd.append(row['CD IMPORT'])
        current_export_cd.append(row['CD EXPORT'])

        current_import.append(row['NVL QTY'])
        current_export.append(row['TP QTY'])

    else:
        if current_code == row['NVL']:
            current_import_cd.append(row['CD IMPORT'])
            current_export_cd.append(row['CD EXPORT'])
            
            current_import.append(row['NVL QTY'])
            current_export.append(row['TP QTY'])
        else:
            # GET DICE FOR DEVIDE LIST
            k = 0
            im = 0
            ex = 0
            dice = [0]
            
            length = len(current_import)
            im = current_import.pop(0)


            for i in range(length):
                ex += current_export[i]
                if ex < im:
                    pass
                else:
                    try:
                        im += current_import.pop(0)
                    except:
                        pass
                    dice.append(i)
            # --------------------------
            clean_import_cd = [x for x in current_import_cd if x]
            temp_cd = []

            for i in range(len(dice)):
                try:
                    temp_cd.append(clean_import_cd.pop(0))
                except IndexError:
                    break

                if i < len(dice) - 1:
                    temp_cd += [None] * (dice[i+1] - dice[i] - 1)


            if len(temp_cd) < len(current_export):
                temp_cd.append(None)
            
            to_delete += range(count-(len(current_export) - len(temp_cd)), count)
            import_cds += temp_cd

            current_import = []
            current_export = []
            current_import_cd = []
            current_export_cd = []

            current_import_cd.append(row['CD IMPORT'])
            current_export_cd.append(row['CD EXPORT'])
            current_import.append(row['NVL QTY'])
            current_export.append(row['TP QTY'])

            

    current_code = row['NVL']



dfy = dfx.drop(to_delete)
dfy['CD IMPORT'] = import_cds
dfy['CD EXPORT'] = export_cds
dfy['NVL NAME'] = nvl_names
dfy['NVL UNIT'] = nvl_units
dfy['IMPORT_DATE'] = import_dates
dfy['NVL QTY'] = import_qty
dfy['Tổng trị giá'] = tax_totals
dfy['Thuế suất XNK'] = tax_rates
dfy['EXPORT_DATE'] = export_dates
dfy['TP QTY'] = export_qty
dfy['TP'] = tps
dfy['TP UNIT'] = tp_units
dfy['TP NAME'] = tp_names
dfy['bom_value'] = bom_values



In [ ]:
dfy.shape[0]

In [ ]:
df = pd.read_excel('../output/kok.xlsx', converters={'CD IMPORT': str, 'CD EXPORT': str})

current_code = None
current_import = []
current_export = []
current_import_cd = []
current_nvl_names = []
current_nvl_units = []
current_import_cds = []
current_import_dates = []
current_import_qty = []
current_tax_totals = []
current_tax_rates = []


nvl_names = []
nvl_units = []
import_cds = []
import_dates = []
import_qty = []
tax_totals = []
tax_rates = []


export_cds = []
export_dates = []
export_qty = []
tps = []
tp_units = []
tp_names = []
bom_values = []

to_delete = []


for count, row in df.iterrows():
    if not current_code:
        current_import_cd.append(row['CD IMPORT'])
        current_import.append(row['NVL QTY'])
        current_export.append(row['TP QTY'])
        current_nvl_names.append(row['NVL NAME'])
        current_nvl_units.append(row['NVL UNIT'])
        current_import_dates.append(row['IMPORT_DATE'])
        current_import_qty.append(row['NVL QTY'])
        current_tax_totals.append(row['Tổng trị giá'])
        current_tax_rates.append(row['Thuế suất XNK'])

    else:
        if current_code == row['NVL']:
            current_import_cd.append(row['CD IMPORT'])
            current_import.append(row['NVL QTY'])
            current_export.append(row['TP QTY'])
            current_nvl_names.append(row['NVL NAME'])
            current_nvl_units.append(row['NVL UNIT'])
            current_import_dates.append(row['IMPORT_DATE'])
            current_import_qty.append(row['NVL QTY'])
            current_tax_totals.append(row['Tổng trị giá'])
            current_tax_rates.append(row['Thuế suất XNK'])

        else:
            # GET DICE FOR DEVIDE LIST
            k = 0
            im = 0
            ex = 0
            dice = [0]
            
            length = len(current_import)
            im = current_import.pop(0)

            for i in range(length):
                ex += current_export[i]
                if ex < im:
                    pass
                else:
                    
                    try:
                        im += current_import.pop(0)
                        dice.append(i)
                    except:
                        dice.append(i-1)
                    
            # --------------------------
            clean_import_cd = [x for x in current_import_cd if x]
            clean_nvl_names = [x for x in current_nvl_names if x]
            clean_nvl_units = [x for x in current_nvl_units if x]
            clean_import_dates = [x for x in current_import_dates if x]
            clean_import_qty = [x for x in current_import_qty if x]
            clean_tax_totals = [x for x in current_tax_totals if x]
            clean_tax_rates = [x for x in current_tax_rates if x]

            temp_cd = []
            temp_nvl_names = []
            temp_nvl_units = []
            temp_import_dates = []
            temp_import_qty = []
            temp_tax_totals = []
            temp_tax_rates = []


            for i in range(len(dice)):
                try:
                    temp_cd.append(clean_import_cd.pop(0))
                    temp_nvl_names.append(current_nvl_names.pop(0))
                    temp_nvl_units.append(current_nvl_units.pop(0))
                    temp_import_dates.append(current_import_dates.pop(0))
                    temp_import_qty.append(current_import_qty.pop(0))
                    temp_tax_totals.append(current_tax_totals.pop(0))
                    temp_tax_rates.append(current_tax_rates.pop(0))
                except IndexError:
                    break

                if i < len(dice) - 1:
                    temp_cd += [None] * (dice[i+1] - dice[i] - 1)
                    temp_nvl_names += [None] * (dice[i+1] - dice[i] - 1)
                    temp_nvl_units += [None] * (dice[i+1] - dice[i] - 1)
                    temp_import_dates += [None] * (dice[i+1] - dice[i] - 1)
                    temp_import_qty += [None] * (dice[i+1] - dice[i] - 1)
                    temp_tax_totals += [None] * (dice[i+1] - dice[i] - 1)
                    temp_tax_rates += [None] * (dice[i+1] - dice[i] - 1)


            if len(temp_cd) < len(current_export):
                temp_cd.append(None)
                temp_nvl_names.append(None)
                temp_nvl_units.append(None)
                temp_import_dates.append(None)
                temp_import_qty.append(None)
                temp_tax_totals.append(None)
                temp_tax_rates.append(None)
            
            to_delete += range(count-(len(current_export) - len(temp_cd)), count)
            import_cds += temp_cd
            nvl_names += temp_nvl_names
            nvl_units += temp_nvl_units
            import_dates += temp_import_dates
            import_qty += temp_import_qty
            tax_totals += temp_tax_totals
            tax_rates += temp_tax_rates


            current_import = []
            current_export = []
            current_import_cd = []
            current_nvl_names = []
            current_nvl_units = []
            current_import_cds = []
            current_import_dates = []
            current_import_qty = []
            current_tax_totals = []
            current_tax_rates = []


            current_import_cd.append(row['CD IMPORT'])
            current_import.append(row['NVL QTY'])
            current_export.append(row['TP QTY'])
            current_nvl_names.append(row['NVL NAME'])
            current_nvl_units.append(row['NVL UNIT'])
            current_import_dates.append(row['IMPORT_DATE'])
            current_import_qty.append(row['NVL QTY'])
            current_tax_totals.append(row['Tổng trị giá'])
            current_tax_rates.append(row['Thuế suất XNK'])


    current_code = row['NVL']

import_cds.pop()
nvl_names.pop()
nvl_units.pop()
import_dates.pop()
import_qty.pop()
tax_totals.pop()
tax_rates.pop()

df['CD IMPORT'] = import_cds
df['NVL NAME'] = nvl_names
df['NVL UNIT'] = nvl_units
df['IMPORT_DATE'] = import_dates
df['NVL QTY'] = import_qty
df['Tổng trị giá'] = tax_totals
df['Thuế suất XNK'] = tax_rates




In [ ]:
df = pd.read_excel('../output/res.xlsx')

In [ ]:
df.columns

In [ ]:
[
'NVL', 'NVL NAME', 'NVL UNIT', 'IMPORT_DATE', 'CD IMPORT', 'CD EXPORT',
'NVL QTY', 'TP QTY', 'TPxBOM', 'TP', 'TP NAME', 'TP UNIT', 'bom_value',
'Tổng trị giá', 'Thuế suất XNK', 'EXPORT_DATE'
]